In [1]:
#| default_exp classes.DomoPDP

In [2]:
#| exporti
import asyncio
import datetime as dt
import io
import json
from dataclasses import dataclass, field
from enum import Enum, auto

from fastcore.basics import patch_to

import httpx
import pandas as pd

#from ..utils.chunk_execution import chunk_list
#from ..utils.DictDot import DictDot
import domolibrary.utils.DictDot as util_dd
#from . import DomoCertification as dmdc
#from .DomoAuth import DomoDeveloperAuth, DomoFullAuth
import domolibrary.client.DomoAuth as dmda
#from .routes import pdp_routes
import domolibrary.routes.pdp as pdp_routes
import domolibrary.client.DomoError as de
import domolibrary.classes.DomoDataset as dmd



### PDP Policy Parameter

In [3]:
#| export
class PDP_Parameter:
    column_name: str
    column_values_ls: list
    operator: str = 'EQUALS' or 'GREATER_THAN' or 'LESS_THAN' or 'GREATER_THAN_EQUAL' or 'LESS_THAN_EQUAL' or 'BETWEEN'
    ignore_case: bool = True
    type: str = 'COLUMN' or 'DYNAMIC' #column sets parameter on data vs dynamic creates on Domo Trusted Attribute


In [4]:
#| export
@patch_to(PDP_Parameter)
def generate_parameter_simple(obj):

        return pdp_routes.generate_policy_parameter_simple(column_name=obj.name,
                                                           type=obj.type,
                                                           column_values_ls=obj.values,
                                                           operator=obj.operator,
                                                           ignore_case=obj.ignoreCase
                                                           )

### PDP Policy

In [5]:
#| export
@dataclass
class PDP_Policy:
    dataset_id: str
    filter_group_id: str
    name: str
    #resources: list
    parameters_ls: list[dict]
    user_ls: list[str]
    group_ls: list[str]
    virtual_user_ls: list[str]

    @classmethod
    def _from_json(cls, obj):
        dd = util_dd.DictDot(obj)

        return cls(dataset_id=dd.dataSourceId,
                   filter_group_id=dd.filterGroupId,
                   name=dd.name,
                   #resources=dd.resources,
                   parameters_ls=dd.parameters,
                   user_ls=dd.userIds,
                   group_ls=dd.groupIds,
                   virtual_user_ls=dd.virtualUserIds)

    @classmethod
    async def upsert_policy(cls, 
                            auth: dmda.DomoAuth,
                            dataset_id: str,
                            policy_definition: dict, # body sent to the API (uses camelCase instead of snake_case)
                            debug_api: bool = False,
                            debug_prn: bool = False
                            ):
        
        #print(policy_definition)
        policy_id = policy_definition.get('filterGroupId')
        if policy_id:
            if debug_prn:
                print(f'Updating policy {policy_id} in {auth.domo_instance}')    
            res = await pdp_routes.update_policy(auth=auth,
                                                 dataset_id=dataset_id,
                                                 policy_id=policy_id,
                                                 body=policy_definition,
                                                 debug_api=debug_api)
            return res
        else:
            if debug_prn:
                print(f'Policy does not exist. Creating policy in {auth.domo_instance}')
            res = await pdp_routes.create_policy(auth=auth,
                                                 dataset_id=dataset_id,
                                                 body=policy_definition,
                                                 debug_api=debug_api)
            return res
    

In [6]:
#| export
@patch_to(PDP_Policy)
def add_parameter(self: PDP_Policy, parameter_obj):
    self.parameters.concat(parameter_obj)

In [7]:
#| export
@patch_to(PDP_Policy)
def generate_body(
        self: PDP_Policy, 
        #params: list[dict] = ''
        ):
        self.parameters_ls = [PDP_Parameter.generate_parameter_simple(param) for param in self.parameters_ls]
        return pdp_routes.generate_policy_body(policy_name=self.name,
                                               dataset_id=self.dataset_id,
                                               parameters_ls=self.parameters_ls,
                                               policy_id=self.filter_group_id,
                                               user_ids=self.user_ls,
                                               group_ids=self.group_ls,
                                               virtual_user_ids=self.virtual_user_ls)

### Dataset PDP Policies 

In [8]:
#| export
class Dataset_PDP_Policies:

    dataset: dmd.DomoDataset  # domo dataset class
    policies: list[PDP_Policy] = None  

    def __init__(self, dataset):
        self.dataset = dataset
        self.policies = []

### Get PDP Policies from dataset

In [9]:
#| export
@patch_to(Dataset_PDP_Policies)
async def get_policies(
        self: Dataset_PDP_Policies, 
        auth: dmda.DomoAuth = None, 
        dataset_id: str = None, 
        return_raw: bool = False,
        debug_api: bool = False
    ):

        dataset_id = dataset_id or self.dataset.id
        auth = auth or self.dataset.auth

        res = await pdp_routes.get_pdp_policies(auth=auth, dataset_id=dataset_id, debug_api=debug_api)

        if return_raw:
              return res

        if res.status == 200:
            domo_policy = [PDP_Policy._from_json(
                policy_obj) for policy_obj in res.response]
            self.policies = domo_policy
            return domo_policy

### sample implementation of get_policies

In [10]:
import os
import domolibrary.classes.DomoDataset as dmd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

# instantiate dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=os.environ["DOJO_DATASET_ID"]) 
# instantiate dataset_pdp_policies
dataset_pdp_policies = Dataset_PDP_Policies(dataset=dataset)
# get policies
await dataset_pdp_policies.get_policies(return_raw=False)

[PDP_Policy(dataset_id='04c1574e-c8be-4721-9846-c6ffa491144b', filter_group_id=1149, name='Test Update Policy Name', parameters_ls=[DictDot(name='objectID', value='000003007', values=['000003007'], type='COLUMN', operator='EQUALS', not=False, ignoreCase=True)], user_ls=[1893952720], group_ls=None, virtual_user_ls=None)]

### sample implementation of generate_body

In [11]:
import os
import domolibrary.classes.DomoDataset as dmd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

# instantiate dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=os.environ["DOJO_DATASET_ID"]) 

# instantiate dataset_pdp_policies
dataset_pdp_policies = Dataset_PDP_Policies(dataset=dataset)

await dataset_pdp_policies.get_policies(return_raw=False)

# print([policy.name for policy in dataset_pdp_policies.policies])

match_policy = dataset_pdp_policies.policies[0]

match_policy.generate_body()


{'name': 'Test Update Policy Name',
 'dataSourceId': '04c1574e-c8be-4721-9846-c6ffa491144b',
 'userIds': [1893952720],
 'virtualUserIds': [],
 'groupIds': [],
 'dataSourcePermissions': False,
 'parameters': [{'type': 'COLUMN',
   'name': 'objectID',
   'values': ['000003007'],
   'operator': 'EQUALS',
   'ignoreCase': True}],
 'filterGroupId': 1149}

### sample implementation of upsert_policy

In [12]:
new_param = {'type': 'COLUMN', 'name': 'Country', 'values': ['Japan'], 'operator': 'EQUALS', 'ignoreCase': True}

body['parameters'].append(new_param)


dataset_id = body['dataSourceId']
policy_id = body['filterGroupId']

await PDP_Policy.upsert_policy(
    auth=token_auth,
    dataset_id=dataset_id,
    policy_definition=body,
    debug_api=False,
    debug_prn=False
)


NameError: name 'body' is not defined

### Search PDP Policies by name and ID

In [ ]:
#| export
class SearchPDP_NotFound(de.DomoError):
    def __init__(self, 
                 domo_instance,
                 dataset_id,
                 message='not found',
                 function_name='search_pdp'):

        super().__init__(domo_instance=domo_instance, entity_id=dataset_id, message=message, function_name=function_name)
         
@patch_to(Dataset_PDP_Policies, cls_method=True)
async def search_pdp_policies(
    cls: Dataset_PDP_Policies,
    auth: dmda.DomoAuth,
    search: str,
    dataset_id: str = None,
    search_method: str = 'id' or 'name',
    is_exact_match: bool = True,
    return_raw: bool = False, 
    debug_api: bool = False,
    session: httpx.AsyncClient = None
):
    
    all_pdp_policies = await Dataset_PDP_Policies(cls).get_policies(
        auth = auth,
        dataset_id = dataset_id,
        debug_api=debug_api
    )
    
    if return_raw:
        return all_pdp_polcies

    if search_method == 'name':
        if is_exact_match:
            policy_search = next((policy for policy in all_pdp_policies if policy.name == search), None)
            #print(policy_search)   
            
            if not policy_search:
                raise SearchPDP_NotFound(
                    dataset_id=dataset_id,
                    message=f'There is no policy named "{search}" on dataset_id {dataset_id}',
                    domo_instance=auth.domo_instance
                )  
            
            return policy_search
        else:
            policy_search = [policy for policy in all_pdp_policies if search.lower() in policy.name.lower()]
            if not policy_search:
                raise SearchPDP_NotFound(
                    dataset_id=dataset_id,
                    message=f'There is no policy name containing "{search}" on dataset_id {dataset_id}',
                    domo_instance=auth.domo_instance
                )  
            
            return policy_search
    else:
        policy_search = next((policy for policy in all_pdp_policies if policy.filter_group_id == search), None)
         
    if not policy_search:
        raise SearchPDP_NotFound(
            dataset_id=dataset_id,
            message=f'There is no policy id "{search}" on dataset_id {dataset_id}',
            domo_instance=auth.domo_instance
        )  
          
    return policy_search    
    

### sample implementation of search_pdp_policies

In [ ]:
import os
import domolibrary.classes.DomoDataset as dmd

dataset_id = os.environ["DOJO_DATASET_ID"]

#authenticate into instance
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

#instance of dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=dataset_id)

#instance of pdp_policies for the dataset
dpp = Dataset_PDP_Policies(dataset=dataset)

#pull all policies for that dataset
await dpp.get_policies()

#search policies based on name
try:
    policy = await dpp.search_pdp_policies(
        auth=token_auth, 
        dataset_id=dataset.id, 
        search='Test Policy Name Employee 123', 
        search_method='name', 
        is_exact_match=True, 
        debug_api=False
    )
    print(policy)

except SearchPDP_NotFound as e:
    print(e)

search_pdp: 04c1574e-c8be-4721-9846-c6ffa491144b :There is no policy named "Test Policy Name Employee 123" on dataset_id 04c1574e-c8be-4721-9846-c6ffa491144b at domo-community


### Delete PDP Policy

In [ ]:
#| export
@patch_to(PDP_Policy)
async def delete_policy(
    self: PDP_Policy, 
    auth: dmda.DomoAuth, 
    policy_id: str = None,
    dataset_id: str = None, 
    debug_api: bool = False
):

    dataset_id = dataset_id or self.dataset_id
    policy_id = policy_id or self.filter_group_id

    res = await pdp_routes.delete_policy(auth=auth, dataset_id=dataset_id, policy_id=policy_id, debug_api=debug_api)
    
    return res

#### Sample implementation of delete_policy

In [ ]:
import os
import domolibrary.classes.DomoDataset as dmd

dataset_id = os.environ["DOJO_DATASET_ID"]

#authenticate into instance
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

#instance of dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=dataset_id)

#instance of pdp_policies for the dataset
dpp = Dataset_PDP_Policies(dataset=dataset)

policy = None
#search policies based on name
try:
    policy = await dpp.search_pdp_policies(
        auth=token_auth, 
        dataset_id=dataset.id, 
        search='Test Policy Name Employee 123', 
        search_method='name', 
        is_exact_match=True, 
        debug_api=False
    )
    print(policy)

except SearchPDP_NotFound as e:
    print(e)

#delete the policy based on filtergroup id
if policy:
    await policy.delete_policy(auth=token_auth, debug_api=True) #don't need to include policy or dataset since those are already tied with policy class

search_pdp: 04c1574e-c8be-4721-9846-c6ffa491144b :There is no policy named "Test Policy Name Employee 123" on dataset_id 04c1574e-c8be-4721-9846-c6ffa491144b at domo-community


### Enable/Disable PDP on dataset

In [ ]:
#| export
@patch_to(Dataset_PDP_Policies, cls_method=True)
async def toggle_dataset_pdp(
    cls: Dataset_PDP_Policies,
    auth: dmda.DomoAuth,
    dataset_id: str = None,
    is_enable: bool = True, # True will enable pdp, False will disable pdp
    debug_api: bool = False,
    session: httpx.AsyncClient = None
):

    return await pdp_routes.toggle_pdp(
        auth=auth,
        dataset_id=dataset_id,
        is_enable=is_enable,
        debug_api=debug_api,
        session=session
    )



### sample implementation of generate_body

In [ ]:
selected_policy = next(
    (policy for policy in dataset_pdp_policies.policies if policy.name == 'Test Policy Name Employee 123'), None)
# print(selected_policy)
if selected_policy:
    body = selected_policy.generate_body()
    print(body)

### sample implementation of upsert_policy

In [ ]:
selected_policy = next(
    (policy for policy in dataset_pdp_policies.policies if policy.name == 'Test Policy Name Employee 123'), None)
# print(selected_policy)

if selected_policy:
    body = selected_policy.generate_body()
    print(body)

    new_param = {'type': 'COLUMN', 'name': 'Country', 'values': ['Japan'], 'operator': 'EQUALS', 'ignoreCase': True}

    body['parameters'].append(new_param)


    dataset_id = body['dataSourceId']
    policy_id = body['filterGroupId']

    res = await PDP_Policy.upsert_policy(
        auth=token_auth,
        dataset_id=dataset_id,
        policy_definition=body,
        debug_api=True,
        debug_prn=True
    )


### Search PDP Policies by name and ID

In [ ]:
#| export
class SearchPDP_NotFound(de.DomoError):
    def __init__(self, 
                 domo_instance,
                 dataset_id,
                 message='not found',
                 function_name='search_pdp'):

        super().__init__(domo_instance=domo_instance, entity_id=dataset_id, message=message, function_name=function_name)
         
@patch_to(Dataset_PDP_Policies, cls_method=True)
async def search_pdp_policies(
    cls: Dataset_PDP_Policies,
    auth: dmda.DomoAuth,
    search: str,
    dataset_id: str = None,
    search_method: str = 'id' or 'name',
    is_exact_match: bool = True,
    return_raw: bool = False, 
    debug_api: bool = False,
    session: httpx.AsyncClient = None
):
    
    all_pdp_policies = await Dataset_PDP_Policies(cls).get_policies(
        auth = auth,
        dataset_id = dataset_id,
        debug_api=debug_api
    )
    
    if return_raw:
        return all_pdp_polcies

    if search_method == 'name':
        if is_exact_match:
            policy_search = next((policy for policy in all_pdp_policies if policy.name == search), None)
            #print(policy_search)   
            
            if not policy_search:
                raise SearchPDP_NotFound(
                    dataset_id=dataset_id,
                    message=f'There is no policy named "{search}" on dataset_id {dataset_id}',
                    domo_instance=auth.domo_instance
                )  
            
            return policy_search
        else:
            policy_search = [policy for policy in all_pdp_policies if search.lower() in policy.name.lower()]
            if not policy_search:
                raise SearchPDP_NotFound(
                    dataset_id=dataset_id,
                    message=f'There is no policy name containing "{search}" on dataset_id {dataset_id}',
                    domo_instance=auth.domo_instance
                )  
            
            return policy_search
    else:
        policy_search = next((policy for policy in all_pdp_policies if policy.filter_group_id == search), None)
         
    if not policy_search:
        raise SearchPDP_NotFound(
            dataset_id=dataset_id,
            message=f'There is no policy id "{search}" on dataset_id {dataset_id}',
            domo_instance=auth.domo_instance
        )  
          
    return policy_search    
    

### sample implementation of search_pdp_policies

In [ ]:
import os
import domolibrary.classes.DomoDataset as dmd

dataset_id = os.environ["DOJO_DATASET_ID"]

#authenticate into instance
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

#instance of dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=dataset_id)

#instance of pdp_policies for the dataset
dpp = Dataset_PDP_Policies(dataset=dataset)

#pull all policies for that dataset
await dpp.get_policies()

#search policies based on name
try:
    policy = await dpp.search_pdp_policies(
        auth=token_auth, 
        dataset_id=dataset.id, 
        search='Test Policy Name Employee 123', 
        search_method='name', 
        is_exact_match=True, 
        debug_api=False
    )
    policy

except SearchPDP_NotFound as e:
    print(e)

search_pdp: 04c1574e-c8be-4721-9846-c6ffa491144b :There is no policy named "Test Policy Name Employee 123" on dataset_id 04c1574e-c8be-4721-9846-c6ffa491144b at domo-community


### Enable/Disable PDP on dataset

In [ ]:
#| export
@patch_to(Dataset_PDP_Policies, cls_method=True)
async def toggle_dataset_pdp(
    cls: Dataset_PDP_Policies,
    auth: dmda.DomoAuth,
    dataset_id: str = None,
    is_enable: bool = True, # True will enable pdp, False will disable pdp
    debug_api: bool = False,
    session: httpx.AsyncClient = None
):

    return await pdp_routes.toggle_pdp(
        auth=auth,
        dataset_id=dataset_id,
        is_enable=is_enable,
        debug_api=debug_api,
        session=session
    )



### sample implementation of toggle_dataset_pdp

In [ ]:
import os
import domolibrary.classes.DomoDataset as dmd

dataset_id = os.environ["DOJO_DATASET_ID"]

#authenticate into instance
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

#instance of dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=dataset_id)

#instance of pdp_policies for the dataset
dpp = Dataset_PDP_Policies(dataset=dataset)

#toggle pdp on dataset
await dpp.toggle_dataset_pdp(auth=token_auth, dataset_id=dataset.id, is_enable=True, debug_api=False)

In [ ]:
# | hide
import nbdev
nbdev.nbdev_export()
